In [1]:
#Dependencies

import pandas as pd
import re

In [2]:
#Store filepaths in variables

endangered_data = 'endangered_status_data.csv'
structure_data = 'language_structure_data.csv'
time_data = 'deaths_data.csv'

In [3]:
#Based on researching the endangered languages database, the CSV columns are as follows: 
#1) Location_code
#2) code_val 
#3) Primary Name 
#4) alternate_names
#5) endangerment_level 
#6) num_speakers
#7) classification
#8) dialect_varieties
#9) Notes?
#10) Public_comment
#11) locations_places
#12) Region?
#13) Coordinates

#Read in DF number 1: Endangerment of languages.

endangered_df = pd.read_csv(endangered_data,
                names=['Location Code', 'Code Value', 'Language', 'Alternate Names', 'Endangerment Level',
                       'Number of Speakers', 'Classification', 'Dialect Varieties', 'Notes', 'Public Comment', 
                       'Location Places', 'Region', 'Coordinates'])



In [4]:
#Drop unnecessary columns

final_endangered_df = endangered_df.drop(['Location Code', 'Code Value', 'Alternate Names', 'Classification', 'Dialect Varieties',
                   'Notes', 'Public Comment', 'Location Places'], axis = 1)


In [5]:
#Get every set of coordinates in a separate column for each language 
coordinates_df = endangered_df.Coordinates.str.split(";", expand=True)


In [6]:
#There are up to 7 different coordinate locations per language. 
#Rename coordinates columns from 0-6 to "Coord 1 - Coord 7"

coords_renamed_df = coordinates_df.rename(index=str, columns=
                      {0: "coord_primary",
                      1: "coord_secondary",
                      2: "Coord 3",
                      3: "Coord 4",
                      4: "Coord 5",
                      5: "Coord 6",
                      6: "Coord 7"})

In [7]:
#Pull out the primary coordinates for each language and separate them into different columns

primary_coords_df = coords_renamed_df.coord_primary.str.split(",", n= -1, expand=True)
primary_coords_df = primary_coords_df.rename(columns={
                        0: "Primary Latitude",
                        1: "Primary Longitude"})


lat_col = primary_coords_df['Primary Latitude'].tolist()
lon_col = primary_coords_df['Primary Longitude'].tolist()

In [8]:
final_endangered_df['Latitude'] = lat_col
final_endangered_df['Longitude'] = lon_col

final_en_df = final_endangered_df


In [9]:

#Remove the "endangerement level" commentary by splitting the string at the parentheses

final_en_df['Endangerment Level'] = final_en_df['Endangerment Level'].str.split("(",
        n = -1, expand=True)

#Clean up the Number of Speakers column by splitting ranges and taking just the lowest number
final_en_df['Number of Speakers'] = final_en_df['Number of Speakers'].str.split("-", n=-1, expand=True)

#Delete all the weird characters and stuff out of Number of Speeakers column
final_en_df['Number of Speakers'] = final_en_df['Number of Speakers'].str.replace('\W', "")

#Drop the coordinates column that has multiple and confusing lon/lats 
#final_en_df = final_en_df.drop(['Coordinates'], axis=1)

final_en_df.head()


,Language,Endangerment Level,Number of Speakers,Region,Coordinates,Latitude,Longitude
0,!Xun,Vulnerable,14000,Africa,"-28.74358,23.983154; -17.560247, 18.050537; -1...",-28.74358,23.983154
1,'Ôrôê,Endangered,590,Pacific,"-21.4223,165.4678",-21.4223,165.4678
2,(Lower) Tanana,Critically Endangered,25,North America,"65.157778, -149.37;64.521111, -146.980556;64.5...",65.157778,-149.37
3,Aasáx,Dormant,0,Africa,"-5.1948,37.738",-5.1948,37.738
4,Abaga,Critically Endangered,5,Pacific,"-6.1883,145.5976",-6.1883,145.5976


In [10]:
#THINGS TO DO:

#Import structure and time csv files

#Clean structure data if necessary 

#Join structure and time data to the final_en_df on 'Language'

#Export all data into a giant CSV file

#JSONify all data into a giant JSON file

In [11]:
#Read in the strucure data, only take in the columns we want, rename the columns so we
#can join on language and distinguish which lats/lons are from which dataset
structure_df = pd.read_csv(structure_data)
structure_df = structure_df[['Name', 'latitude', 'longitude', 'genus',
                            'family', 'countrycodes']].rename(columns={
    'Name': 'Language',
    'latitude': 'lat_str',
    'longitude': 'lon_str',})

structure_df.head()



,Language,lat_str,lon_str,genus,family,countrycodes
0,Arapesh (Abu),-3.450000,142.950000,Kombio-Arapesh,Torricelli,PG
1,Aari,6.000000,36.583333,South Omotic,Afro-Asiatic,ET
2,Abau,-4.000000,141.250000,Upper Sepik,Sepik,PG
3,Arabic (Chadian),13.833333,20.833333,Semitic,Afro-Asiatic,TD
4,Abidji,5.666667,-4.583333,Kwa,Niger-Congo,CI


In [12]:
#INNER MERGE on Language --> 668 records were able to merge on 
#Language from the structures dataset and the endangered dataset
struc_en_df = pd.merge(structure_df, final_en_df, how='inner', on="Language")
struc_en_df.head()


,Language,lat_str,lon_str,genus,family,countrycodes,Endangerment Level,Number of Speakers,Region,Coordinates,Latitude,Longitude
0,Abau,-4.000000,141.250000,Upper Sepik,Sepik,PG,Threatened,7000,Pacific,"-3.97, 141.32",-3.97,141.32
1,Alabama,32.333333,-87.416667,Muskogean,Muskogean,US,Endangered,250,North America,"30.7158,-94.7028",30.7158,-94.7028
2,Arbore,5.000000,36.750000,Lowland East Cushitic,Afro-Asiatic,ET,Threatened,4441,Africa,"5.0,36.75",5.0,36.75
3,Abun,-0.500000,132.500000,North-Central Bird's Head,West Papuan,ID,Threatened,3000,Southeast Asia,"-0.52, 132.19; -0.66, 131.91",-0.52,132.19
4,Abui,-8.250000,124.666667,Greater Alor,Timor-Alor-Pantar,ID,Threatened,16000,Southeast Asia,NaN,NaN,NaN


In [13]:
#Export to csv file 
struc_en_df.to_csv(r'raw data', index=False)

In [14]:
#Read in time data we want (century and language so we can merge on language)
time_df = pd.read_csv(time_data)
time_df = time_df[['Language', 'Century of Death']]


In [20]:
#Merge time data with structure_data ONLY
#In theory, endangered status data and dead language data should be mutually exclusive

death_structure_df = pd.merge(structure_df, time_df, how="inner", on="Language")

In [23]:
#Export to csv file --> There are only 75 records that match
death_structure_df.to_csv(r'raw data', index=False)